## Assignemnt 2

Given the personal details of a patient, we have to predict medical cost. <p>

Columns:

<li>age: age of primary beneficiary 

<li>sex: insurance contractor gender, female, male 

<li>bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height, 
objective index of body weight (kg/m^2) using the ratio of height to weight, ideally 18.5 to 24.9 

<li>children: Number of children covered by health insurance / Number of dependents 

<li>smoker: Smoking 

<li>region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest. 

<li>charges: Individual medical costs billed by health insurance 

#### Hadoop Commands

1. ./allstart.sh <B>
2. pysparknb (to open jupyter notebook) 
3. Copy Files using Bitwise SSH Client 
4. hadoopfs -copyFromLocal filename.csv    
    

The given dataset can be used by primarily two types of users: <p>
    <li> Insurance Companies
    <li> Consumers</p>
        <p> <b> Insurance Companies </b>
            <li> Companies can use consumer data and offer technology, consulting, and other services to providers, payors, government agencies, and life science organizations.
            <li> Create targeted marketing campaigns that will acquire new customers.
            <li> Identify possible issues, based on client’s data, and furthermore put them into a suitable risk class.     
            <li> Data-driven decision making into areas such as cross-selling and reducing customer churn. 
            <li> Spotlight the highest-value clients and high-potential leads so agents can invest resources more efficiently, predict customer churn more accurately to help improve retention.
            <li> Insurers can compare a person's data against past fraudulent profiles and identify cases that require more investigation.     
            <li> Insurers can offer usage-based policies and determine claims liability easily and accurately.
            <li> Become more personalized by taking into account the medical history of a customer along with their habits. The data can also be utilized to decide a pricing model that fits into the budget of the client and also is profitable for the company.     
         <p> <b> Consumers </b>
            <li> Consumers who agree to let insurance companies track their habits can learn more about themselves, while insurers can use the data to influence behavior and reduce risks.
            <li> They can identify various premium plans that are in accordance with their lifestyle and budget.

In [1]:
#Entry point to programming Spark with the Dataset and DataFrame API.
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName('nlp').getOrCreate()
from pyspark.sql.functions import monotonically_increasing_id, desc, row_number
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.master("local[*]").config("spark.executor.memory", "70g").config("spark.driver.memory", "50g").config("spark.memory.offHeap.enabled",True).config("spark.memory.offHeap.size","16g").appName("sampleCodeForReference").getOrCreate()

In [3]:
#Garbage Collector
import gc
gc.collect()

148

In [4]:
#Read the data

df=spark.read.csv('insurance.csv', header= True, inferSchema=True)
df= df.withColumn('id', row_number().over(Window.orderBy(monotonically_increasing_id())))

In [5]:
#Rows and Columns in the dataset

df.show()

+---+------+------+--------+------+---------+-----------+---+
|age|   sex|   bmi|children|smoker|   region|    charges| id|
+---+------+------+--------+------+---------+-----------+---+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|  1|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|  2|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|  3|
| 33|  male|22.705|       0|    no|northwest|21984.47061|  4|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|  5|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|  6|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|  7|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|  8|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|  9|
| 60|female| 25.84|       0|    no|northwest|28923.13692| 10|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208| 11|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251| 12|
| 23|  male|  34.4|       0|    no|southwest|   1826.843| 13|
| 56|fem

In [6]:
#Number of rows and columns

print(df.count(),",", len(df.columns))

1338 , 8


In [7]:
#Column description

df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)
 |-- id: integer (nullable = true)



### Data Exploration & Pre-processing

In [8]:
#Check for duplicates and null values

df=df.dropDuplicates()
print(df.count(),",", len(df.columns))
df=df.na.drop()
print(df.count(),",", len(df.columns))

1338 , 8
1338 , 8


In [9]:
#Column desciption

df.describe()

DataFrame[summary: string, age: string, sex: string, bmi: string, children: string, smoker: string, region: string, charges: string, id: string]

In [10]:
#Data Statistics

df.summary().show()

+-------+------------------+------+------------------+-----------------+------+---------+------------------+------------------+
|summary|               age|   sex|               bmi|         children|smoker|   region|           charges|                id|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+------------------+
|  count|              1338|  1338|              1338|             1338|  1338|     1338|              1338|              1338|
|   mean| 39.20702541106129|  null|30.663396860986538|  1.0949177877429|  null|     null|13270.422265141257|             669.5|
| stddev|14.049960379216147|  null| 6.098186911679012|1.205492739781914|  null|     null|12110.011236693992|386.39164069632767|
|    min|                18|female|             15.96|                0|    no|northeast|         1121.8739|                 1|
|    25%|                27|  null|             26.29|                0|  null|     null|         4738.2

In [11]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,1338,39.20702541106129,14.049960379216147,18,64
sex,1338,None,None,female,male
bmi,1338,30.663396860986538,6.098186911679012,15.96,53.13
children,1338,1.0949177877429,1.205492739781914,0,5
smoker,1338,None,None,no,yes
region,1338,None,None,northeast,southwest
charges,1338,13270.422265141257,12110.011236693992,1121.8739,63770.42801
id,1338,669.5,386.39164069632767,1,1338


In [12]:
df=df.toPandas()

#### Adding Features

In [13]:
import numpy as np

# create a list of our conditions
conditions = [
    (df['bmi'] < 18.5),
    (df['bmi'] >= 18.5) & (df['bmi'] <= 24.9),
    (df['bmi'] >= 25) & (df['bmi'] <= 29.9),
    (df['bmi'] >= 30)
    ]

# create a list of the values we want to assign for each condition
values = ['Underweight', 'Normal', 'Overweight', 'Obese']

# create a new column and use np.select to assign values to it using our lists as arguments
df['bmi_class'] = np.select(conditions, values)

# display updated DataFrame
df.head()

,age,sex,bmi,children,smoker,region,charges,id,bmi_class
0,19,female,27.900,0,yes,southwest,16884.92400,1,Overweight
1,18,male,33.770,1,no,southeast,1725.55230,2,Obese
2,28,male,33.000,3,no,southeast,4449.46200,3,Obese
3,33,male,22.705,0,no,northwest,21984.47061,4,Normal
4,32,male,28.880,0,no,northwest,3866.85520,5,Overweight


In [14]:
# create a list of our conditions
conditions = [
    (df['age'] < 18),
    (df['age'] >= 18) & (df['age'] <= 24),
    (df['age'] >= 25) & (df['age'] <= 45),
    (df['age'] > 45)
    ]

# create a list of the values we want to assign for each condition
values = ['Child', 'Young Adult', 'Middle-Aged', 'Aged']

# create a new column and use np.select to assign values to it using our lists as arguments
df['age_class'] = np.select(conditions, values)

# display updated DataFrame
df.head()

,age,sex,bmi,children,smoker,region,charges,id,bmi_class,age_class
0,19,female,27.900,0,yes,southwest,16884.92400,1,Overweight,Young Adult
1,18,male,33.770,1,no,southeast,1725.55230,2,Obese,Young Adult
2,28,male,33.000,3,no,southeast,4449.46200,3,Obese,Middle-Aged
3,33,male,22.705,0,no,northwest,21984.47061,4,Normal,Middle-Aged
4,32,male,28.880,0,no,northwest,3866.85520,5,Overweight,Middle-Aged


In [15]:
df=spark.createDataFrame(df)

#### Categorical Encoding

In [16]:
from pyspark.ml.feature import OneHotEncoder as ohe
from pyspark.ml.feature import StringIndexer

sex_indexer= StringIndexer(inputCol='sex', outputCol= 'sex_indexer')
bmi_indexer= StringIndexer(inputCol='bmi_class', outputCol= 'bmi_indexer')
age_indexer= StringIndexer(inputCol='age_class', outputCol= 'age_indexer')
region_indexer= StringIndexer(inputCol='region', outputCol= 'region_indexer')
smoker_indexer= StringIndexer(inputCol='smoker', outputCol= 'smoker_indexer')
df=sex_indexer.fit(df).transform(df)
df=smoker_indexer.fit(df).transform(df)
df=region_indexer.fit(df).transform(df)
df=bmi_indexer.fit(df).transform(df)
df=age_indexer.fit(df).transform(df)

sex_ohe= ohe(inputCol='sex_indexer', outputCol= 'sex_ohe')
smoker_ohe= ohe(inputCol='smoker_indexer', outputCol= 'smoker_ohe')
region_ohe= ohe(inputCol='region_indexer', outputCol= 'region_ohe')
bmi_ohe= ohe(inputCol='bmi_indexer', outputCol= 'bmi_ohe')
age_ohe= ohe(inputCol='age_indexer', outputCol= 'age_ohe')
df= sex_ohe.fit(df).transform(df)
df=smoker_ohe.fit(df).transform(df)
df=region_ohe.fit(df).transform(df)
df=bmi_ohe.fit(df).transform(df)
df=age_ohe.fit(df).transform(df)

df.show()

+---+------+------+--------+------+---------+-----------+---+----------+-----------+-----------+--------------+--------------+-----------+-----------+-------------+-------------+-------------+-------------+-------------+
|age|   sex|   bmi|children|smoker|   region|    charges| id| bmi_class|  age_class|sex_indexer|smoker_indexer|region_indexer|bmi_indexer|age_indexer|      sex_ohe|   smoker_ohe|   region_ohe|      bmi_ohe|      age_ohe|
+---+------+------+--------+------+---------+-----------+---+----------+-----------+-----------+--------------+--------------+-----------+-----------+-------------+-------------+-------------+-------------+-------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|  1|Overweight|Young Adult|        1.0|           1.0|           2.0|        1.0|        2.0|    (1,[],[])|    (1,[],[])|(3,[2],[1.0])|(4,[1],[1.0])|    (2,[],[])|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|  2|     Obese|Young Adult|        0.0|           0.0|     

In [17]:
df=df.drop('sex','region','smoker', 'sex_indexer', 'region_indexer', 'smoker_indexer', 'bmi_indexer', 'age_indexer')
df.show()

+---+------+--------+-----------+---+----------+-----------+-------------+-------------+-------------+-------------+-------------+
|age|   bmi|children|    charges| id| bmi_class|  age_class|      sex_ohe|   smoker_ohe|   region_ohe|      bmi_ohe|      age_ohe|
+---+------+--------+-----------+---+----------+-----------+-------------+-------------+-------------+-------------+-------------+
| 19|  27.9|       0|  16884.924|  1|Overweight|Young Adult|    (1,[],[])|    (1,[],[])|(3,[2],[1.0])|(4,[1],[1.0])|    (2,[],[])|
| 18| 33.77|       1|  1725.5523|  2|     Obese|Young Adult|(1,[0],[1.0])|(1,[0],[1.0])|(3,[0],[1.0])|(4,[0],[1.0])|    (2,[],[])|
| 28|  33.0|       3|   4449.462|  3|     Obese|Middle-Aged|(1,[0],[1.0])|(1,[0],[1.0])|(3,[0],[1.0])|(4,[0],[1.0])|(2,[0],[1.0])|
| 33|22.705|       0|21984.47061|  4|    Normal|Middle-Aged|(1,[0],[1.0])|(1,[0],[1.0])|(3,[1],[1.0])|(4,[2],[1.0])|(2,[0],[1.0])|
| 32| 28.88|       0|  3866.8552|  5|Overweight|Middle-Aged|(1,[0],[1.0])|(1,[0],[1

In [18]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: long (nullable = true)
 |-- charges: double (nullable = true)
 |-- id: long (nullable = true)
 |-- bmi_class: string (nullable = true)
 |-- age_class: string (nullable = true)
 |-- sex_ohe: vector (nullable = true)
 |-- smoker_ohe: vector (nullable = true)
 |-- region_ohe: vector (nullable = true)
 |-- bmi_ohe: vector (nullable = true)
 |-- age_ohe: vector (nullable = true)



In [19]:
cols= [c[0] for c in df.dtypes if c[1] != 'string']

In [20]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml import Pipeline

In [21]:
vc= VectorAssembler(inputCols = ['children', 'sex_ohe','smoker_ohe','region_ohe','age_ohe','bmi_ohe'], outputCol= 'feature')

df= vc.transform(df)



In [24]:
df=df.select(['feature', 'charges'])
df.show()

+--------------------+-----------+
|             feature|    charges|
+--------------------+-----------+
|(12,[5,9],[1.0,1.0])|  16884.924|
|(12,[0,1,2,3,8],[...|  1725.5523|
|(12,[0,1,2,3,6,8]...|   4449.462|
|(12,[1,2,4,6,10],...|21984.47061|
|(12,[1,2,4,6,9],[...|  3866.8552|
|(12,[2,3,6,9],[1....|  3756.6216|
|(12,[0,2,3,7,8],[...|  8240.5896|
|(12,[0,2,4,6,9],[...|  7281.5056|
|(12,[0,1,2,6,9],[...|  6406.4107|
|(12,[2,4,7,9],[1....|28923.13692|
|(12,[1,2,6,9],[1....|  2721.3208|
|(12,[3,7,9],[1.0,...| 27808.7251|
|(12,[1,2,5,8],[1....|   1826.843|
|(12,[2,3,7,8],[1....| 11090.7178|
|(12,[1,3,6,8],[1....| 39611.7577|
|(12,[0,1,2,5,10],...|   1837.237|
|(12,[0,2,7,8],[1....| 10797.3362|
|(12,[1,2,10],[1.0...| 2395.17155|
|(12,[1,2,5,7,8],[...|  10602.385|
|(12,[1,5,6,8],[1....|  36837.467|
+--------------------+-----------+
only showing top 20 rows



#### Scaling

In [25]:
ss = MinMaxScaler(inputCol='feature', outputCol='features')

In [28]:
pipe= Pipeline(stages= [ ss])
pipe.fit(df)


PipelineModel_0da5d6f76bf0

In [29]:
# Data splitting
train, test= df.randomSplit([0.8,0.2], seed=123)

In [27]:
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, GBTRegressor, RandomForestRegressor

In [30]:
print(train.count())
print(test.count())

1083
255


#### Linear Regression

In [33]:
lr= LinearRegression(featuresCol='feature', labelCol='charges', maxIter= 25 )
model= lr.fit(train)

In [34]:
print('coefficients: %s' % str(model.coefficients))
print('Intercept: %s' % str(model.intercept))

coefficients: [496.27276201745985,-303.8436728010304,-24334.235129684588,-1129.5381780762286,-51.21377286285491,-1119.0480455052982,2614.437922206722,8343.73372071558,2649.8199702592824,-1253.2853210430226,-2202.6546231017605,-4315.3616512516255]
Intercept: 28118.5111022702


In [35]:
summ= model.summary
print('Iterations: %d' % summ.totalIterations)
print('objectiveHistory %s' % str(summ.objectiveHistory))
summ.residuals.show()
print('RMSE: %f' % summ.rootMeanSquaredError)
print('r2: %f' % summ.r2)

Iterations: 1
objectiveHistory [0.0]
+-------------------+
|          residuals|
+-------------------+
| -958.7533761918157|
| -4144.492438209279|
| -4654.508300226737|
| -5095.580662244197|
| 3013.5270171692273|
|-1408.2022367181362|
|-2846.0978987355957|
|-2195.5054833983686|
| 15062.660232566708|
|-3771.4345539850938|
| -3957.016643422653|
|-2420.3194434226525|
|  9756.331556577348|
| -2515.851217457571|
| 1102.7339598971148|
| 420.98509787965213|
|  672.9459978796522|
| 12374.959347879652|
|   1876.64454993839|
|-4093.7912799140486|
+-------------------+
only showing top 20 rows

RMSE: 6175.815276
r2: 0.746509


In [36]:
pred=model.transform(test)
pred.select('prediction','charges','feature').show()

+------------------+-----------+--------------------+
|        prediction|    charges|             feature|
+------------------+-----------+--------------------+
| 8607.697538209279|  7731.4271|(12,[0,1,2,3,6,8]...|
|13840.720574700674| 11946.6259|(12,[0,1,2,3,7,8]...|
| 1593.881562682789|  1719.4363|(12,[0,1,2,3,9],[...|
| 9686.021943422653| 3877.30425|(12,[0,1,2,4,6,8]...|
| 5286.643890102885|  4894.7533|(12,[0,1,2,4,6,9]...|
| 4337.274588044147|  5125.2157|(12,[0,1,2,4,6,10...|
| 2219.109427854888| 2803.69785|(12,[0,1,2,4,10],...|
| -389.870362312442| 2775.19215|(12,[0,1,2,4,11],...|
| 8618.187670780211|   7740.337|(12,[0,1,2,5,6,8]...|
|  9114.46043279767|   4260.744|(12,[0,1,2,5,6,8]...|
|  9114.46043279767|   5253.524|(12,[0,1,2,5,6,8]...|
| 5211.355141495365|   3906.127|(12,[0,1,2,5,6,9]...|
| 9948.105415969301|  10436.096|(12,[0,1,2,5,7,9]...|
| 9240.962954268045| 4667.60765|(12,[0,1,2,6,8],[...|
| 5834.130424983203|  6406.4107|(12,[0,1,2,6,9],[...|
|15962.804276811825|11488.31

In [37]:
from pyspark.ml.evaluation import RegressionEvaluator

evalu= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='r2' )
print('R squared= %g' %evalu.evaluate(pred))

evalu1= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='rmse' )
print("rmse: %g" % evalu1.evaluate(pred))

R squared= 0.715035
rmse: 6080.53


#### Gradient Boosting Trees 

In [38]:
gbt= GBTRegressor(featuresCol='feature', labelCol='charges', maxIter= 35 )
model= gbt.fit(train)
pred=model.transform(test)
pred.select('prediction','charges','feature').show()

+------------------+-----------+--------------------+
|        prediction|    charges|             feature|
+------------------+-----------+--------------------+
| 7586.900267521125|  7731.4271|(12,[0,1,2,3,6,8]...|
|11722.666218566865| 11946.6259|(12,[0,1,2,3,7,8]...|
|1501.6930165722945|  1719.4363|(12,[0,1,2,3,9],[...|
| 8346.154314996125| 3877.30425|(12,[0,1,2,4,6,8]...|
| 6189.682071815749|  4894.7533|(12,[0,1,2,4,6,9]...|
| 6672.760334934017|  5125.2157|(12,[0,1,2,4,6,10...|
|11008.423223095066| 2803.69785|(12,[0,1,2,4,10],...|
|2954.5721176040292| 2775.19215|(12,[0,1,2,4,11],...|
| 7759.265355582049|   7740.337|(12,[0,1,2,5,6,8]...|
| 6607.658466084202|   4260.744|(12,[0,1,2,5,6,8]...|
| 6607.658466084202|   5253.524|(12,[0,1,2,5,6,8]...|
| 5974.832970920326|   3906.127|(12,[0,1,2,5,6,9]...|
|13801.280384133808|  10436.096|(12,[0,1,2,5,7,9]...|
| 6706.203482957495| 4667.60765|(12,[0,1,2,6,8],[...|
|10234.229854928068|  6406.4107|(12,[0,1,2,6,9],[...|
|13883.751303470717|11488.31

In [39]:
from pyspark.ml.evaluation import RegressionEvaluator

evalu= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='r2' )
print('R squared= %g' %evalu.evaluate(pred))

evalu1= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='rmse' )
print("rmse: %g" % evalu1.evaluate(pred))

R squared= 0.814463
rmse: 4906.37


#### Decision Tree

In [40]:
dt= DecisionTreeRegressor(featuresCol='feature', labelCol='charges' )
model= dt.fit(train)
pred=model.transform(test)
pred.select('prediction','charges','feature').show()

+------------------+-----------+--------------------+
|        prediction|    charges|             feature|
+------------------+-----------+--------------------+
| 7709.028154963504|  7731.4271|(12,[0,1,2,3,6,8]...|
|12264.090589058822| 11946.6259|(12,[0,1,2,3,7,8]...|
|2877.1080386419753|  1719.4363|(12,[0,1,2,3,9],[...|
| 7709.028154963504| 3877.30425|(12,[0,1,2,4,6,8]...|
| 6559.596719082569|  4894.7533|(12,[0,1,2,4,6,9]...|
| 6559.596719082569|  5125.2157|(12,[0,1,2,4,6,10...|
|       11057.39639| 2803.69785|(12,[0,1,2,4,10],...|
|2877.1080386419753| 2775.19215|(12,[0,1,2,4,11],...|
| 7709.028154963504|   7740.337|(12,[0,1,2,5,6,8]...|
| 7709.028154963504|   4260.744|(12,[0,1,2,5,6,8]...|
| 7709.028154963504|   5253.524|(12,[0,1,2,5,6,8]...|
| 7709.028154963504|   3906.127|(12,[0,1,2,5,6,9]...|
|14865.524260909091|  10436.096|(12,[0,1,2,5,7,9]...|
| 6559.596719082569| 4667.60765|(12,[0,1,2,6,8],[...|
| 7709.028154963504|  6406.4107|(12,[0,1,2,6,9],[...|
|14086.585834029853|11488.31

In [41]:
from pyspark.ml.evaluation import RegressionEvaluator

evalu= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='r2' )
print('R squared= %g' %evalu.evaluate(pred))

evalu1= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='rmse' )
print("rmse: %g" % evalu1.evaluate(pred))

R squared= 0.839198
rmse: 4567.64


#### Random Forest

In [42]:
rf= RandomForestRegressor(featuresCol='feature', labelCol='charges' )
model= rf.fit(train)
pred=model.transform(test)
pred.select('prediction','charges','feature').show()

+------------------+-----------+--------------------+
|        prediction|    charges|             feature|
+------------------+-----------+--------------------+
|  7750.89385410375|  7731.4271|(12,[0,1,2,3,6,8]...|
|12408.653266775418| 11946.6259|(12,[0,1,2,3,7,8]...|
| 4276.038690222232|  1719.4363|(12,[0,1,2,3,9],[...|
|  8487.51861612924| 3877.30425|(12,[0,1,2,4,6,8]...|
| 6747.621499065448|  4894.7533|(12,[0,1,2,4,6,9]...|
| 7129.558465663858|  5125.2157|(12,[0,1,2,4,6,10...|
|   6594.1934622801| 2803.69785|(12,[0,1,2,4,10],...|
| 4577.195473773203| 2775.19215|(12,[0,1,2,4,11],...|
| 7985.188151370705|   7740.337|(12,[0,1,2,5,6,8]...|
| 7965.911079099862|   4260.744|(12,[0,1,2,5,6,8]...|
| 7965.911079099862|   5253.524|(12,[0,1,2,5,6,8]...|
|7751.5910529742205|   3906.127|(12,[0,1,2,5,6,9]...|
| 12200.47076503634|  10436.096|(12,[0,1,2,5,7,9]...|
| 7707.851989735756| 4667.60765|(12,[0,1,2,6,8],[...|
| 7528.654549065434|  6406.4107|(12,[0,1,2,6,9],[...|
|13425.196958331397|11488.31

In [43]:
from pyspark.ml.evaluation import RegressionEvaluator

evalu= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='r2' )
print('R squared= %g' %evalu.evaluate(pred))

evalu1= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='rmse' )
print("rmse: %g" % evalu1.evaluate(pred))

R squared= 0.821342
rmse: 4814.56


#### Model Tuning: Decision Tree

In [44]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [45]:
grid= (ParamGridBuilder()
       .addGrid(dt.maxDepth, [2, 5, 10, 20, 30 ])
       .addGrid(dt.maxBins, [10,20,30])
       .build())

In [46]:
dteval= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='rmse')

In [47]:
dtcv= CrossValidator(estimator= dt, estimatorParamMaps= grid, evaluator= dteval, numFolds= 4)
dtc=dtcv.fit(train)

In [48]:
pred= dtc.transform(test)

In [49]:
evalu= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='r2' )
print('R squared= %g' %evalu.evaluate(pred))
print("rmse: %g" % dteval.evaluate(pred))

R squared= 0.839198
rmse: 4567.64


In [50]:
grid1= (ParamGridBuilder()
       .addGrid(gbt.maxDepth, [2, 5, 10, 20, 30 ])
       .addGrid(gbt.maxBins, [10,20,30])
        .addGrid(gbt.maxIter, [10,20,25, 30,35, 50])
       .build())

#### Model Tuning: Random Forest

In [51]:
from pyspark.ml.evaluation import RegressionEvaluator

gbteval= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='rmse' )

In [52]:
gbtcv= CrossValidator(estimator= rf, estimatorParamMaps= grid1, evaluator= gbteval, numFolds= 4)
gbtc=gbtcv.fit(train)

In [53]:
pred= gbtc.transform(test)

In [54]:
evalu= RegressionEvaluator(predictionCol='prediction', labelCol='charges', metricName='r2' )
print('R squared= %g' %evalu.evaluate(pred))
print("rmse: %g" % gbteval.evaluate(pred))

R squared= 0.821342
rmse: 4814.56
